In [1]:
import networkx as nx
import scipy.sparse as sp
import numpy as np
import math
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
raw_edge_list = open('/content/drive/My Drive/Colab Notebooks/DSLab/yeast.edgelist','r')
G = nx.Graph()
G = nx.parse_edgelist(raw_edge_list, delimiter='\t', create_using=G,nodetype=str, data=(('weight', float),))
# Get graph edges and nodes from networkx graph object
nodes = G.nodes
edges = G.edges

print("Graph's Nodes : {} / Edges : {}".format(len(nodes), len(edges)))

Graph's Nodes : 6526 / Edges : 532180


In [6]:
G.remove_edges_from(nx.selfloop_edges(G))

In [7]:
nx.adjacency_matrix(G).todense()

matrix([[0, 1, 1, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [8]:
!pip install stellargraph

     |████████████████████████████████| 440kB 8.0MB/s 


In [9]:
from stellargraph import StellarGraph
from stellargraph.data import EdgeSplitter

In [10]:
es_test = EdgeSplitter(G)
graph_test, node_pairs, link_labels_test = es_test.train_test_split(
    p=0.25, method="global"
)

** Sampled 132623 positive and 132623 negative edges. **


In [11]:
from sklearn.model_selection import train_test_split
# Do the same process to compute a training subset from within the test graph
es_train = EdgeSplitter(graph_test, G)
graph_train, examples_train, link_labels_train = es_train.train_test_split(
    p=0.25, method="global"
)
examples_train,examples_val, link_labels_train, link_label_val = train_test_split(examples_train,link_labels_train,test_size=0.1)

** Sampled 99468 positive and 99468 negative edges. **


## Find node embeddings using GCN

In [12]:
!pip install dgl

     |████████████████████████████████| 3.0MB 8.3MB/s 


In [13]:
import dgl
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [14]:
from dgl.nn.pytorch.conv import GraphConv

In [16]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GraphConv(in_feats=6526,out_feats=502)
        self.layer2 = GraphConv(in_feats=502,out_feats=2)

    def forward(self, g, features):
        x = F.relu(self.layer1(g, features))
        x = self.layer2(g, x)
        return x
net = Net()
print(net)

Net(
  (layer1): GraphConv(in=6526, out=502, normalization=both, activation=None)
  (layer2): GraphConv(in=502, out=2, normalization=both, activation=None)
)


In [ ]:
features = np.identity(len(graph_train.nodes))

In [ ]:
features.shape

(6526, 6526)

In [ ]:
dgl_graph_train = DGLGraph()
dgl_graph_train.from_networkx(graph_train)

In [ ]:
generator = sg.mapper.PaddedGraphGenerator(graphs)

In [ ]:
def graph_distance(graph1, graph2):
    spec1 = nx.laplacian_spectrum(graph1.to_networkx(feature_attr=None))
    spec2 = nx.laplacian_spectrum(graph2.to_networkx(feature_attr=None))
    k = min(len(spec1), len(spec2))
    return np.linalg.norm(spec1[:k] - spec2[:k])

In [ ]:
net.train()
logits = net(dgl_graph_train, th.FloatTensor(features))

In [ ]:
emb = logits.detach().numpy()

In [ ]:
def return_embedding_of_node(node_name):
    #nodes are maintained in adjacency matrix in the same order as G.nodes
    nodes = np.asarray(G.nodes)
    index = np.where(nodes==node_name)
    return emb[index]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler


# 1. link embeddings
def link_examples_to_features(link_examples, binary_operator):
    feat =  [binary_operator(return_embedding_of_node(src), return_embedding_of_node(dst))
             for src, dst in link_examples]
    features = np.asarray(feat)
    features = np.reshape(features,(features.shape[0],features.shape[2]))
    return features
    


# 2. training classifier
def train_link_prediction_model(link_examples, link_labels, binary_operator):
    clf = link_prediction_classifier()
    link_features = link_examples_to_features(link_examples, binary_operator)
    clf.fit(link_features, link_labels)
    return clf


def link_prediction_classifier(max_iter=2000):
    lr_clf = LogisticRegressionCV(Cs=10, cv=10, scoring="roc_auc", max_iter=max_iter)
    return Pipeline(steps=[("sc", StandardScaler()), ("clf", lr_clf)])


# 3. and 4. evaluate classifier
def evaluate_link_prediction_model(clf, link_examples_test, link_labels_test, binary_operator):
    link_features_test = link_examples_to_features(link_examples_test, binary_operator)
    score = evaluate_roc_auc(clf, link_features_test, link_labels_test)
    return score


def evaluate_roc_auc(clf, link_features, link_labels):
    predicted = clf.predict_proba(link_features)

    # check which class corresponds to positive links
    positive_column = list(clf.classes_).index(1)
    return roc_auc_score(link_labels, predicted[:, positive_column])

In [ ]:
def operator_hadamard(u, v):
    return u * v


def operator_l1(u, v):
    return np.abs(u - v)


def operator_l2(u, v):
    return (u - v) ** 2


def operator_avg(u, v):
    return (u + v) / 2.0

def run_link_prediction(binary_operator):
    clf = train_link_prediction_model(examples_train, link_labels_train, binary_operator)
    score = evaluate_link_prediction_model(
        clf,
        examples_val,
        link_label_val,
        binary_operator,
    )
    return {
        "classifier": clf,
        "binary_operator": binary_operator,
        "score": score,
    }


binary_operators = [operator_hadamard, operator_l1, operator_l2, operator_avg]

In [ ]:
results = [run_link_prediction(op) for op in binary_operators]

In [ ]:
import pandas as pd
best_result = max(results, key=lambda result: result["score"])

print(f"Best result from '{best_result['binary_operator'].__name__}'")

pd.DataFrame(
    [(result["binary_operator"].__name__, result["score"]) for result in results],
    columns=("name", "ROC AUC score"),
).set_index("name")

Best result from 'operator_avg'


,ROC AUC score
name,
operator_hadamard,0.547837
operator_l1,0.524061
operator_l2,0.517573
operator_avg,0.554613


In [ ]:
test_score = evaluate_link_prediction_model(
    best_result["classifier"],
    node_pairs,
    link_labels_test,
    best_result["binary_operator"],
)
print(
    f"ROC AUC score on test set using '{best_result['binary_operator'].__name__}': {test_score}"
)

ROC AUC score on test set using 'operator_avg': 0.56039620926023


In [ ]:
#Graph Visualization

def visualize_graph(file_name, graph):
    title = file_name
    plt.figure(figsize=(20,20))
    node_size = 15
    node_alpha = .6

    edge_color = '#483D8B'
    edge_alpha = 0.15
    edge_style = 'dashed'
    edge_width = 2

    protein_node_color = "#F08080"
    rna_node_color = "#1E90FF"

    pos = nx.drawing.nx_agraph.graphviz_layout(graph)
    nx.draw_networkx_nodes(graph, pos, node_size=node_size, node_color=rna_node_color, alpha=node_alpha)
    nx.draw_networkx_edges(graph, pos, width=edge_width, alpha=edge_alpha)
    plt.axis('off')
    plt.savefig("images/{}.png".format(title))
    plt.title("{} / Blue - Protein".format(title))
    plt.show()

In [ ]:
adj = nx.adjacency_matrix(G)
adj.todense()

In [ ]:
#Remove self-edges
adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
adj.eliminate_zeros()
adj.todense()

In [ ]:
def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

In [ ]:
a = sparse_to_tuple(adj)[0]
np.random.shuffle(a)

In [ ]:
#Number of edges required in train, test and validation set
num_edges = a.shape[0]
num_val_edges = math.floor(num_edges * 0.02)
num_test_edges = math.floor(num_edges * 0.18)
num_train_edges = num_edges - (num_val_edges + num_test_edges)

In [ ]:
#put 80% of the edges into train set, 18% into test set and 2% in validation set
train_edges_true = a[:num_train_edges]
test_edges_true = a[num_train_edges:]

In [ ]:
#prepare validation set
val_edges_true = test_edges_true[:num_val_edges]
test_edges_true = test_edges_true[num_val_edges:]

In [ ]:
test_edges_true

In [ ]:
def sample_non_edges(num_of_edges,true_edges,adjmat):
    adj_dense = adjmat.todense()
    uniq_nodes = np.unique(true_edges)
    edges_false = []
    while(len(edges_false) < num_of_edges):
        rand_ind = np.random.choice(uniq_nodes, 1,replace=True)
        row_in_adj = adj_dense[rand_ind]
        non_edges = np.where(row_in_adj[0] == row_in_adj.min())[1]
        for i in non_edges:
            if i in uniq_nodes:
                arr = [rand_ind[0],i]
                edges_false.append(arr)
        np.delete(uniq_nodes,rand_ind)
    return edges_false

In [ ]:
train_edges_false = sample_non_edges(num_train_edges,train_edges_true,adj)
test_edges_false = sample_non_edges(num_test_edges,test_edges_true,adj)
val_edges_false = sample_non_edges(num_val_edges,val_edges_true,adj)

In [ ]:
train_G = nx.Graph()
train_G.

Following cells are also doing partitioning of the dataset but by dividing the edges array rather picking them up from adjacency matrix. I will train GCN model for both approaches and will see which one would be the best split.

In [ ]:
#Prepare train graph
random_train_edge_ind = np.random.choice(arr_edges.shape[0],num_train_edges,replace=False)
train_arr = arr_edges[random_train_edge_ind,:]
g_train = nx.Graph()
g_train.add_edges_from(train_arr)
visualize_graph("",g_train)

In [ ]:
#Prepare test graph
random_test_edge_ind = np.random.choice(arr_edges.shape[0],num_test_edges,replace=False)
test_arr = arr_edges[random_test_edge_ind,:]
g_test = nx.Graph()
g_test.add_edges_from(test_arr)
visualize_graph("",g_test)

In [ ]:
#prepare validation graph
random_val_edge_ind = np.random.choice(arr_edges.shape[0],num_val_edges,replace=False)
val_arr = arr_edges[random_val_edge_ind,:]
g_val = nx.Graph()
g_val.add_edges_from(val_arr)
visualize_graph("",g_val)

In [ ]:
plt.imshow(adj, cmap='hot', interpolation='nearest')
plt.show()
